## Différentaition automatique.

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aderdouri/EiCNAM/blob/master/Tutorials/computational_graphs.ipynb) [![Open In Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/aderdouri/EiCNAM/blob/master/Tutorials/computational_graphs.ipynb)

### Formula:
$$
z = \cos\left(a_0 + \exp(a_1)\right)\left(\sin(a_2) + \cos(a_3)\right) + (a_1)^{\frac{3}{2}} + a_3
$$


In [ ]:
import tensorflow as tf

# Define the variables
variables = {
    "a0": tf.Variable(1.0, dtype=tf.float32),
    "a1": tf.Variable(2.0, dtype=tf.float32),
    "a2": tf.Variable(3.0, dtype=tf.float32),
    "a3": tf.Variable(4.0, dtype=tf.float32),
}

# Define the function
def func(a0, a1, a2, a3):
    return tf.cos(a0 + tf.exp(a1)) * (tf.sin(a2) + tf.cos(a3)) + tf.pow(a1, 1.5) + a3

# Calculate the gradient
with tf.GradientTape() as tape:
    tape.watch(list(variables.values()))  # Watch the variables
    z = func(*variables.values())

# Compute gradients
grads = tape.gradient(z, list(variables.values()))

# Display gradients as a dictionary
gradient_dict = {name: float(grad.numpy()) for name, grad in zip(variables.keys(), grads)}
print("Computed Gradients:")
for var, grad in gradient_dict.items():
    print(f"{var}: {grad:.6f}")


2024-12-02 17:29:47.455877: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Computed Gradients:
a0: 0.440888
a1: 5.379070
a2: 0.504802
a3: 0.614102


### Black Formula

### Black-Scholes Formula

The Black-Scholes formula for the price of a European call option is given by:

$$
C(S, t) = S\Phi(d_1) - Ke^{-r(T-t)}\Phi(d_2),
$$

where:

$$
d_1 = \frac{\ln(S/K) + \left(r + \frac{\sigma^2}{2}\right)(T-t)}{\sigma\sqrt{T-t}}, \quad
d_2 = d_1 - \sigma\sqrt{T-t}.
$$

#### Parameters:
- C(S, t)\): Call option price at time \(t\),
- \(S\): Current price of the underlying asset,
- \(K\): Strike price of the option,
- \(r\): Risk-free interest rate,
- \(\sigma\): Volatility of the underlying asset,
- \(T\): Time to maturity,
- \($\Phi$($\cdot$)\): Cumulative distribution function of the standard normal distribution.


In [2]:
import tensorflow as tf
import tensorflow_probability as tfp
import math

# Define the Black-Scholes formula as a function
def black_scholes(S, K, r, sigma, T, option_type="call"):
    """
    S: Current stock price
    K: Strike price
    r: Risk-free rate
    sigma: Volatility
    T: Time to maturity
    option_type: "call" or "put"
    """
    d1 = (tf.math.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * tf.sqrt(T))
    d2 = d1 - sigma * tf.sqrt(T)

    if option_type == "call":
        price = S * tfp.distributions.Normal(0.0, 1.0).cdf(d1) - K * tf.exp(-r * T) * tfp.distributions.Normal(0.0, 1.0).cdf(d2)
    elif option_type == "put":
        price = K * tf.exp(-r * T) * tfp.distributions.Normal(0.0, 1.0).cdf(-d2) - S * tfp.distributions.Normal(0.0, 1.0).cdf(-d1)
    else:
        raise ValueError("Invalid option_type. Choose 'call' or 'put'.")
    return price

# Parameters
S = tf.Variable(100.0)  # Stock price
K = tf.Variable(102.0)  # Strike price
r = tf.Variable(0.05)   # Risk-free rate
sigma = tf.Variable(0.2)  # Volatility
T = tf.Variable(1.0)    # Time to maturity

# Calculate the derivatives (Greeks) using TensorFlow's GradientTape
with tf.GradientTape(persistent=True) as tape:
    tape.watch([S, K, r, sigma, T])  # Watch all inputs
    price = black_scholes(S, K, r, sigma, T)

# Compute the Greeks
delta = tape.gradient(price, S)   # Sensitivity to stock price
vega = tape.gradient(price, sigma)  # Sensitivity to volatility
theta = tape.gradient(price, T)   # Sensitivity to time to maturity
rho = tape.gradient(price, r)     # Sensitivity to risk-free rate

# Display the results
print(f"Option Price: {price.numpy():.6f}")
print(f"Delta (∂C/∂S): {delta.numpy():.6f}")
print(f"Vega (∂C/∂σ): {vega.numpy():.6f}")
print(f"Theta (∂C/∂T): {theta.numpy():.6f}")
print(f"Rho (∂C/∂r): {rho.numpy():.6f}")


ImportError: This version of TensorFlow Probability requires TensorFlow version >= 2.18; Detected an installation of version 2.16.2. Please upgrade TensorFlow to proceed.